In [1]:
import pandas as pd
import olga.load_model as load_model
import olga.generation_probability as pgen
import os

In [2]:
df = pd.read_csv('../data/output_data/exp_AB_wide4.csv', index_col=0)

In [3]:
df.head()

,junction_aa,junction,v_call,j_call,decombinator_id,ID,control,PCR,chain,proportion_-3,...,total_-2,total_-1,total_0,total_1,total_2,total_3,total_4,total_14,max_timepoint,max_timepoint_class
1,CAARAIGNTPLVF,TGTGCAGCAAGGGCCATCGGAAACACACCTCTTGTCTTT,TRAV29/DV5,TRAJ29,"23, 17, 2, 8, GGCCATC",17,True,NaN,alpha,NaN,...,NaN,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_1,und
2,CAASIWNYGGSQGNLIF,TGTGCAGCAAGTATATGGAATTATGGAGGAAGCCAAGGAAATCTCA...,TRAV13-1,TRAJ42,"6, 31, 0, 1, TATG",17,True,NaN,alpha,NaN,...,NaN,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_2,und
3,CAASNQAGTALIF,TGTGCAGCAAGTAACCAGGCAGGAACTGCTCTGATCTTT,TRAV13-1,TRAJ15,"6, 5, 0, 3,",17,True,NaN,alpha,NaN,...,NaN,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_2,und
4,CACASKIIF,TGTGCTTGTGCTTCCAAGATAATCTTT,TRAV24,TRAJ3,"18, 18, 5, 11, TT",17,True,NaN,alpha,NaN,...,NaN,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_2,und
5,CACGNTGFQKLVF,TGTGCTTGCGGGAACACAGGCTTTCAGAAACTTGTATTT,TRAV2,TRAJ8,"13, 48, 8, 1, TGCGG",17,True,NaN,alpha,NaN,...,NaN,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_1,und


In [4]:
generative_model = load_model.GenerativeModelVDJ()
marginals_file_name = '../../OLGA/olga/default_models/human_T_beta/model_marginals.txt'
generative_model.load_and_process_igor_model(marginals_file_name)

params_file_name = '../../OLGA/olga/default_models/human_T_beta/model_params.txt'
V_anchor_pos_file ='../../OLGA/olga/default_models/human_T_beta/V_gene_CDR3_anchors.csv'
J_anchor_pos_file = '../../OLGA/olga/default_models/human_T_beta/J_gene_CDR3_anchors.csv'
genomic_data = load_model.GenomicDataVDJ()
genomic_data.load_igor_genomic_data(params_file_name, V_anchor_pos_file, J_anchor_pos_file)

In [5]:
pgen_b = pgen.GenerationProbabilityVDJ(generative_model, genomic_data)

In [6]:
generative_model = load_model.GenerativeModelVJ()
marginals_file_name = '../../OLGA/olga/default_models/human_T_alpha/model_marginals.txt'
generative_model.load_and_process_igor_model(marginals_file_name)

params_file_name = '../../OLGA/olga/default_models/human_T_alpha/model_params.txt'
V_anchor_pos_file ='../../OLGA/olga/default_models/human_T_alpha/V_gene_CDR3_anchors.csv'
J_anchor_pos_file = '../../OLGA/olga/default_models/human_T_alpha/J_gene_CDR3_anchors.csv'
genomic_data = load_model.GenomicDataVJ()
genomic_data.load_igor_genomic_data(params_file_name, V_anchor_pos_file, J_anchor_pos_file)

In [7]:
pgen_a = pgen.GenerationProbabilityVJ(generative_model, genomic_data)

In [8]:
print(df)

            junction_aa                                           junction  \
1         CAARAIGNTPLVF            TGTGCAGCAAGGGCCATCGGAAACACACCTCTTGTCTTT   
2     CAASIWNYGGSQGNLIF  TGTGCAGCAAGTATATGGAATTATGGAGGAAGCCAAGGAAATCTCA...   
3         CAASNQAGTALIF            TGTGCAGCAAGTAACCAGGCAGGAACTGCTCTGATCTTT   
4             CACASKIIF                        TGTGCTTGTGCTTCCAAGATAATCTTT   
5         CACGNTGFQKLVF            TGTGCTTGCGGGAACACAGGCTTTCAGAAACTTGTATTT   
...                 ...                                                ...   
9529     CASSPGPKYNEQFF         TGTGCCAGCAGCCCGGGACCGAAGTACAATGAGCAGTTCTTC   
9530    CASSQDVLAKNIQYF      TGTGCCAGCAGCCAAGACGTCCTAGCCAAAAACATTCAGTACTTC   
9531     CASSQEEGSSEAFF         TGTGCCAGCAGCCAAGAGGAGGGATCCTCTGAAGCTTTCTTT   
9532    CASSQVTLSNAEAFF      TGTGCCAGCAGCCAAGTTACGTTGAGCAACGCTGAAGCTTTCTTT   
9533  CASSTPNGLAAGDTQYF  TGTGCCAGCTCGACCCCAAATGGACTAGCGGCGGGAGATACGCAGT...   

          v_call   j_call                         decombinator_

In [9]:
df['pgen'] = ''

In [10]:
dfa = df.loc[df['chain'] == 'alpha']
dfb = df.loc[df['chain'] == 'beta']

In [11]:
dfa['pgen'] = dfa.apply(lambda row: 
                        pgen_a.compute_aa_CDR3_pgen(row['junction_aa'], 
                                row['v_call'], print_warnings= True), axis=1)
dfb['pgen'] = dfb.apply(lambda row: 
                        pgen_b.compute_aa_CDR3_pgen(row['junction_aa'], 
                        row['v_call'], print_warnings= True), axis=1)
# TRBV1, TRBV17, TRBV21-1, TRBV23-1, TRBV5-3, TRBV5-7, TRBV6-7, TRBV7-5 are pseudogenes that decombinator does not clean up

/tmp/ipykernel_1497382/1919900756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa['pgen'] = dfa.apply(lambda row:


Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Using default V_usage_mask
Unfamiliar V gene/allele: TRBV21-1
No recognized V genes/alleles. Usin

/tmp/ipykernel_1497382/1919900756.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb['pgen'] = dfb.apply(lambda row:


In [12]:
dfa.loc[dfa['pgen'] == 0]

,junction_aa,junction,v_call,j_call,decombinator_id,ID,control,PCR,chain,proportion_-3,...,total_-1,total_0,total_1,total_2,total_3,total_4,total_14,max_timepoint,max_timepoint_class,pgen
24,CASETSGSRLTF,TGTGCTTCTGAAACCAGTGGCTCTAGGTTGACCTTT,TRAV12-3,TRAJ58,"5, 57, 8, 5, TTCT",17,True,NaN,alpha,NaN,...,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_0,und,0.0
55,CAVRSARQLTF,TGTGCTGTTCGTTCTGCAAGGCAACTGACCTTT,TRAV8-2,TRAJ22,"39, 11, 10, 9, CTGTTC",17,True,NaN,alpha,NaN,...,NaN,85012.0,90223.0,78873.0,65517.0,65456.0,NaN,proportion_2,und,0.0
68,CQPLGQNFVF,TGTCAACCCTTGGGTCAGAATTTTGTCTTT,TRAV3,TRAJ26,"24, 14, 13, 11, CAACCCTTG",35,False,2.0,alpha,NaN,...,NaN,NaN,NaN,34950.0,NaN,NaN,73037.0,NaN,und,0.0
70,CVVSDQTGANNLFF,TGTGTTGTGAGTGATCAAACTGGGGCAAACAACCTCTTCTTT,TRAV8-2,TRAJ36,"39, 24, 0, 0,",35,False,2.0,alpha,NaN,...,NaN,NaN,NaN,34950.0,NaN,NaN,73037.0,NaN,und,0.0
74,CVVSVRGATADMRF,TGTGTTGTGAGTGTACGGGGGGCCACGGCTGACATGCGCTTT,TRAV8-2,TRAJ43,"39, 32, 1, 10, TACGGGGGGCCACGGC",35,False,2.0,alpha,NaN,...,NaN,NaN,NaN,34950.0,NaN,NaN,73037.0,NaN,und,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3900,CAVPDILTGGGNKLTF,TGTGCTGTTCCCGATATACTCACGGGAGGAGGAAACAAACTCACCTTT,TRAV3,TRAJ10,"24, 0, 8, 0, TCCCGAT",371,False,0.0,alpha,NaN,...,NaN,91999.0,60696.0,NaN,71521.0,83993.0,44870.0,NaN,und,0.0
3921,CACGVLHC,TGTGCGTGCGGTGTGCTGCATTGC,TRAV13-2,TRAJ35,"7, 54, 8, 15, GTGCGG",373,False,0.0,alpha,NaN,...,NaN,160288.0,98933.0,NaN,159120.0,NaN,98300.0,NaN,und,0.0
3983,CAPERGVSF,TGTGCCCCTGAAAGGGGGGTCAGCTTT,TRAV2,TRAJ20,"13, 9, 9, 18, CCCTGAAAGGGGGG",373,False,0.0,alpha,NaN,...,NaN,160288.0,98933.0,NaN,159120.0,NaN,98300.0,NaN,und,0.0
4028,CAVSGSNYGQNFVF,TGTGCTGTGAGTGGGTCCAACTATGGTCAGAATTTTGTCTTT,TRAV8-2,TRAJ26,"39, 14, 10, 5, CTGTGAGTGGGTCC",373,False,0.0,alpha,NaN,...,NaN,160288.0,98933.0,NaN,159120.0,NaN,98300.0,NaN,und,0.0


In [13]:
dfa.to_csv('../data/output_data/exp_A_wide5_pgen.csv')
dfb.to_csv('../data/output_data/exp_B_wide5_pgen.csv')

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [15]:
dfa_melt = pd.melt(dfa[['pgen', 'max_timepoint_class', 'chain']], id_vars = ['max_timepoint_class', 'chain'])
dfa_melt['pgen_log'] = np.log10(dfa_melt['value'])
dfa_melt = dfa_melt.loc[dfa_melt['value'] > 0]
print(dfa_melt)

dfb_melt = pd.melt(dfb[['pgen', 'max_timepoint_class', 'chain']], id_vars = ['max_timepoint_class', 'chain'])
dfb_melt['pgen_log'] = np.log10(dfb_melt['value'])
dfb_melt = dfb_melt.loc[dfb_melt['value'] > 0]
print(dfb_melt)

     max_timepoint_class  chain variable         value   pgen_log
0                    und  alpha     pgen  1.684988e-09  -8.773403
1                    und  alpha     pgen  1.796038e-07  -6.745685
2                    und  alpha     pgen  9.067683e-06  -5.042504
3                    und  alpha     pgen  3.348758e-08  -7.475116
4                    und  alpha     pgen  4.041352e-10  -9.393473
...                  ...    ...      ...           ...        ...
4070                 und  alpha     pgen  1.259741e-08  -7.899719
4071                 und  alpha     pgen  8.216401e-07  -6.085318
4072                 und  alpha     pgen  6.149321e-08  -7.211173
4073                 und  alpha     pgen  1.154439e-07  -6.937629
4074                 und  alpha     pgen  2.864361e-11 -10.542972

[3961 rows x 5 columns]
     max_timepoint_class chain variable         value   pgen_log
0                    und  beta     pgen  1.122590e-10  -9.949779
1                    und  beta     pgen  7.886121e-14

/home/marti/miniconda3/envs/mm/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/marti/miniconda3/envs/mm/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
# colormap = {'late':'cyan', 'early':'orange', 'und':'lightgrey'}
# colormap_edge = {'late':'darkcyan', 'early':'darkorange', 'und':'darkgrey'}

# ax = plt.subplot()
# sns.kdeplot(data=dfa_melt.loc[dfa_melt['max_timepoint_class'] == 'und'], 
#             x = 'pgen_log', ax=ax, label = 'und', color=colormap['und'])
# for c in ['early', 'late']:
#     print(c)
#     sns.histplot(data=dfa_melt.loc[dfa_melt['max_timepoint_class'] == c], 
#                 x = 'pgen_log', ax=ax, label = c, color=colormap[c], edgecolor =colormap_edge[c],
#                 binwidth=1, binrange=(-20,-4), stat='proportion', alpha = 0.3)
# plt.legend()
# plt.xlim(-20, -4)
# plt.xlabel(r'log$_{10}$(pgen)')
# plt.ylabel('Proportion of sample')
# plt.show()

In [17]:
# colormap = {'late':'cyan', 'early':'orange', 'und':'lightgrey'}
# colormap_edge = {'late':'darkcyan', 'early':'darkorange', 'und':'darkgrey'}

# ax = plt.subplot()
# sns.kdeplot(data=dfb_melt.loc[dfb_melt['max_timepoint_class'] == 'und'], 
#             x = 'pgen_log', ax=ax, label = 'und', color=colormap['und'])
# for c in ['early', 'late']:
#     print(c)
#     sns.histplot(data=dfb_melt.loc[dfb_melt['max_timepoint_class'] == c], 
#                 x = 'pgen_log', ax=ax, label = c, color=colormap[c], edgecolor =colormap_edge[c],
#                 binwidth=1, binrange=(-16,-4), stat='proportion', alpha = 0.3)
# plt.legend()
# plt.xlim(-20, -4)
# plt.xlabel(r'log$_{10}$(pgen)')
# plt.ylabel('Proportion of sample')
# plt.show()

In [18]:
LCMV = pd.DataFrame()

for f in os.listdir('../data/LCMVdata/'):
    if ('E_' in f) and ('_freq' in f):
        lcmv = pd.read_csv('../data/LCMVdata/' + f, index_col=0).reset_index(names='junction_aa')
        lcmv = pd.melt(lcmv, id_vars = ['junction_aa'])
        lcmv = lcmv.loc[lcmv['value'] > 0]
        lcmv[['epitope', 'condition', 'mouse']] = lcmv['variable'].str.split(' ', expand=True)
        LCMV = pd.concat([LCMV, lcmv]).reset_index(drop=True)

In [19]:
LCMV.head()

,junction_aa,variable,value,epitope,condition,mouse
0,CAAGTGDSDYTF,NP205 LCMV40 M1,0.000076,NP205,LCMV40,M1
1,CACRDRGNQNTLYF,NP205 LCMV40 M1,0.000312,NP205,LCMV40,M1
2,CADRGVQNTLYF,NP205 LCMV40 M1,0.002006,NP205,LCMV40,M1
3,CASADRGQNTLYF,NP205 LCMV40 M1,0.000151,NP205,LCMV40,M1
4,CASADRYNSPLYF,NP205 LCMV40 M1,0.000303,NP205,LCMV40,M1


In [20]:
generative_model = load_model.GenerativeModelVDJ()
marginals_file_name = '../../OLGA/olga/default_models/mouse_T_beta/model_marginals.txt'
generative_model.load_and_process_igor_model(marginals_file_name)

params_file_name = '../../OLGA/olga/default_models/mouse_T_beta/model_params.txt'
V_anchor_pos_file ='../../OLGA/olga/default_models/mouse_T_beta/V_gene_CDR3_anchors.csv'
J_anchor_pos_file = '../../OLGA/olga/default_models/mouse_T_beta/J_gene_CDR3_anchors.csv'
genomic_data = load_model.GenomicDataVDJ()
genomic_data.load_igor_genomic_data(params_file_name, V_anchor_pos_file, J_anchor_pos_file)

In [21]:
pgen_b_mouse = pgen.GenerationProbabilityVDJ(generative_model, genomic_data)

In [22]:
LCMV['pgen'] = LCMV.apply(lambda row: 
                        pgen_b_mouse.compute_aa_CDR3_pgen(row['junction_aa'], 
                        print_warnings = True), axis=1)

In [23]:
LCMV.to_csv('../data/output_data/LCMV_pgen.csv')

In [24]:
LCMV.head()

,junction_aa,variable,value,epitope,condition,mouse,pgen
0,CAAGTGDSDYTF,NP205 LCMV40 M1,0.000076,NP205,LCMV40,M1,2.830681e-08
1,CACRDRGNQNTLYF,NP205 LCMV40 M1,0.000312,NP205,LCMV40,M1,3.241128e-11
2,CADRGVQNTLYF,NP205 LCMV40 M1,0.002006,NP205,LCMV40,M1,2.133456e-09
3,CASADRGQNTLYF,NP205 LCMV40 M1,0.000151,NP205,LCMV40,M1,1.348938e-07
4,CASADRYNSPLYF,NP205 LCMV40 M1,0.000303,NP205,LCMV40,M1,5.485257e-17


In [25]:
LCMV_melt = pd.melt(LCMV[['pgen', 'condition', 'epitope']], id_vars = ['condition', 'epitope'])
LCMV_melt['pgen_log'] = np.log10(LCMV_melt['value'])
LCMV_melt = LCMV_melt.loc[LCMV_melt['value'] > 0]
print(LCMV_melt)

     condition epitope variable         value   pgen_log
0       LCMV40   NP205     pgen  2.830681e-08  -7.548109
1       LCMV40   NP205     pgen  3.241128e-11 -10.489304
2       LCMV40   NP205     pgen  2.133456e-09  -8.670916
3       LCMV40   NP205     pgen  1.348938e-07  -6.870008
4       LCMV40   NP205     pgen  5.485257e-17 -16.260803
...        ...     ...      ...           ...        ...
2330       PBS    GP92     pgen  1.928567e-06  -5.714765
2331       PBS    GP92     pgen  3.475092e-06  -5.459034
2332       PBS    GP92     pgen  1.546273e-05  -4.810714
2333       PBS    GP92     pgen  1.004486e-06  -5.998056
2334       PBS    GP92     pgen  3.860320e-10  -9.413377

[2335 rows x 5 columns]


In [26]:
# colormap = {'LCMV40':'cyan', 'LCMV8':'orange', 'PBS':'lightgrey'}
# colormap_edge = {'LCMV40':'darkcyan', 'LCMV8':'darkorange', 'PBS':'darkgrey'}

# ax = plt.subplot()
# sns.kdeplot(data=LCMV_melt.loc[LCMV_melt['condition'] == 'PBS'], 
#             x = 'pgen_log', ax=ax, label = 'PBS', color=colormap['PBS'])
# for c in ['LCMV40', 'LCMV8']:
#     print(c)
#     sns.histplot(data=LCMV_melt.loc[LCMV_melt['condition'] == c], 
#                 x = 'pgen_log', ax=ax, label = c, color=colormap[c], edgecolor =colormap_edge[c],
#                 binwidth=1, binrange=(-20,-3), stat='proportion', alpha = 0.3)
#     # sns.kdeplot(data=LCMV_melt.loc[LCMV_melt['condition'] == c], 
#     #             x = 'pgen_log', ax=ax, label = c, color=colormap[c])
# plt.legend()
# plt.xlim(-20, -3)
# plt.xlabel(r'log$_{10}$(pgen)')
# plt.ylabel('Proportion of sample')
# plt.show()